In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
import pandas as pd
import scipy.stats as spst
from scipy.optimize import newton
import matplotlib.pyplot as plt
import warnings
import os
import sys
sys.path.insert(sys.path.index('')+1, 'D:/Github/PyFeng')
import pyfeng as pf

# Backet Option

In [7]:
sigma = 0.4*np.ones(4)
spot = np.ones(4)*100
texp=5

In [11]:
m = pf.BsmNdMc(sigma, cor=0.5, rn_seed=1234)
_ = m.simulate(tobs = [texp], n_path=100000)
print(m.n_path)
m.path

100000


array([[[0.15969089, 0.34383065, 0.57141668, 0.42421223],
        [2.81374194, 1.3068322 , 0.78634205, 1.05920794],
        [1.45143979, 9.41924528, 0.71067299, 3.11737835],
        ...,
        [0.47244821, 0.35100675, 0.35379762, 0.5379011 ],
        [0.94114672, 2.27251196, 0.22260395, 0.42029163],
        [0.47742711, 0.19772348, 2.01851295, 1.06908854]]])

In [9]:
_ = m.simulate(tobs = [texp], n_path=10000, store=2)
print(m.n_path)

20000


In [13]:
# varying strikes
payoff = lambda x: np.fmax(np.mean(x, axis=1) - strike, 0)

strikes = np.arange(50, 151, 10)
price = []
for strike in strikes:
    price.append(m.price_european(spot, texp, payoff))
np.array(price)

array([54.760356  , 47.93673613, 41.9808547 , 36.80312288, 32.30556719,
       28.41837774, 25.06332626, 22.15480597, 19.62979424, 17.43613897,
       15.52854283])

In [16]:
# varying forwards
payoff = lambda x: np.fmax(np.mean(x, axis=1) - strike, 0)
strike = 100
price = []
for spot in np.arange(50, 151, 10):
    price.append(m.price_european(spot, texp, payoff))
np.array(price)

array([ 4.50422851,  7.72073036, 11.80479205, 16.67827657, 22.24726644,
       28.41837774, 35.12128286, 42.27834076, 49.81623101, 57.67411975,
       65.81251632])

In [40]:
# varying except sigma1=100%
strike = spot = 100
payoff = lambda x: np.fmax(np.mean(x, axis=1) - strike, 0)
price = []
for sigma1 in np.array([5, 10, 15, 20, 30, 40, 50, 60, 70, 80, 100])/100:
    sigma = sigma1 * np.ones(4)
    sigma[0] = 1
    #print(sigma)
    m = pf.BsmNdMc(sigma, cor=0.5, rn_seed=1234)
    m.simulate(tobs = [texp], n_path=400000)
    price.append(m.price_european(spot, texp, payoff))
np.array(price)

array([18.94495855, 20.45829493, 22.49840945, 24.87785108, 30.11378007,
       35.57949546, 41.05473026, 46.42608331, 51.61861158, 56.59050047,
       65.9322128 ])

# For SABR Model

In [49]:
# You can use BsmNdMc because sigma_t is also a geometric Brownian motion

vov = 0.1
m = pf.BsmNdMc(vov, rn_seed=1234)
tobs = np.arange(0, 101)/100*texp
m.simulate(tobs = tobs, n_path=1000)
print(np.squeeze(m.path))

[[1.         1.         1.         ... 1.         1.         1.        ]
 [1.00118402 0.9983181  1.01856407 ... 1.01792387 0.98656839 1.01310779]
 [1.01765418 0.98166993 0.98295842 ... 0.9652479  0.99062338 1.00845641]
 ...
 [1.35936004 0.70046279 1.08451555 ... 0.54520725 0.89730747 1.06115368]
 [1.34417254 0.70802306 1.10211743 ... 0.53382163 0.86305561 1.10271591]
 [1.41330074 0.67305521 1.0961984  ... 0.53251188 0.84203739 1.12967599]]
